# Scrape username from LinkedIn

In [5]:
#from tqdm.notebook import tqdm
from tqdm import tqdm #check for the scarping progress
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from scrape_linkedin import ProfileScraper
from selenium.webdriver.common.by import By
import pickle

In [2]:
def get_name():
    # get username
    name_list = []
    #driver.page_source is a entire page's HTML
    soup = BeautifulSoup(driver.page_source,'html.parser') # this step get all the static page info, BeautifulSoup is a object
    
    for a in soup.find(class_="search-results__list list-style-none ").find_all('a', href=True):
        name = a['href'].split('/')[-2]  #a['href'] : /in/anastasisbele/
        if a['href'].split('/')[1] == 'in' and name not in name_list:
            name_list.append(name)
    batch(name_list)

In [32]:
#not need for now
# save while scraping
import pickle
def batch(name):
    with open('name.pkl', 'ab') as fp: 
        #with open(file_name, mode), use with makes sure the file is closed after the session, 'a' for append, 'b' for binary mode
        pickle.dump(name, fp) #pickle.dump(obj, file), write object into file

In [3]:
#dynamic scraping
driver = webdriver.Chrome()
url = "https://www.linkedin.com/search/results/people/?keywords=data%20analyst&origin=SWITCH_SEARCH_VERTICAL"
driver.get(url)

#login linkedin
driver.find_element_by_css_selector('body > div > main > p > a').click()
time.sleep(0.5)
driver.find_element_by_css_selector('input#username').send_keys('zz629@cornell.edu') # your email
driver.find_element_by_css_selector('input#password').send_keys('Zx16m73yl') # your password for linkedin
driver.find_element_by_css_selector('#app__container > main > div:nth-child(2) > form > div.login__form_action_container > button').click()

for i in tqdm(range(1)): # how many pages do we want to scrape
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # scroll down and wait website to load
    time.sleep(10)
    
    name_list = []
    get_name()  # get name by using Beautiful soup
    time.sleep(5)

    # click on next bottom to go to next page
    xpath = '//button[contains(@aria-label,"Next")]'
    driver.find_element(By.XPATH, xpath).click()

100%|██████████| 1/1 [00:20<00:00, 20.23s/it]


### Get name id from pickle file

In [6]:
#not need for now
# open name.pkl file
name = []
with open('name.pkl', 'rb') as fr:
    try:
        while True:
            name.append(pickle.load(fr)) 
            #pickle.load(fr): pickle.load(file)
            #read the object from the file and return the reconstituted object hierarchy specified therein
    except EOFError:
        pass

from itertools import chain
name_list = list(chain.from_iterable(name)) #chain thins in different arrays together

### Use write csv to save by rows

In [ ]:
import csv
LI_AT = 'AQEDASqt89oFOYnbAAABcB2vVKkAAAFxeby1ZU0APL5S3jkundKs8NMku9JqGfod9KTZSvenbZTjqIv5LLffYlAmjiNpGiHad6WIlwDbX2-TfVS56hWIcBUB4wW0CZ2y7uIM_BBcikJXup38mpvfiUwA'
# elle's
# LI_AT = 'AQEDASninKgFCQENAAABcWJ8RbkAAAFxhojJuU0ARn_JC06BmSzB8515Xro1AqXinajR4vaRFt5_RFnE1Yc9_6F1VWCAS9BjkPnfqHkirT9j3Mjdb_U0JpJ8EDgfItrqRsndIjAMxyTc0LqheG64WKPJ'

# users = ['yuchen-miranda-zhao', 'jamie-yu-3348434b', 'rongxinz', 'ziyi-li-333a21168'] 

with open('experience.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for name in name_list:
        with ProfileScraper(cookie = LI_AT) as scraper:
            profile = scraper.scrape(user = name)
        for job in profile.experiences['jobs']:
            writer.writerow([name, job['title'], job['company'], job['date_range'], job['location'], job['description']])
            #print(name, job['title'], job['company'], job['date range'], job['location'], job['description'])


### Use following to save simultaneously in two files

In [42]:
import pickle
def info_batch(info):
    with open('info.pkl', 'ab') as fp: 
        pickle.dump(info, fp) 
    
def exp_batch(exp):
    with open('exp.pkl', 'ab') as fp:
        pickle.dump(exp, fp)


In [44]:
for name in users:
    with ProfileScraper(cookie = LI_AT) as scraper:
        profile = scraper.scrape(user = name)
        
    # save to info file        
    summary = profile.personal_info['summary']
    skill = [profile.skills[i]['name'] for i in range(len(profile.skills))]
    certi = profile.accomplishments['certifications']
    course = profile.accomplishments['courses']

    info_batch(tuple([name, summary, skill, course, certi]))
    
    # save to exp file
    for job in profile.experiences['jobs']:
        exp_batch(tuple([name, job['title'], job['company'], job['date_range'], job['location'], job['description']]))
 

### Convert pickle to csv

In [ ]:
# call info pickle file
colnames = ['name', 'summary', 'skill', 'course', 'certi']

import pickle
import pandas as pd
info = []
with open('info.pkl', 'rb') as fr:
    try:
        while True:
            info.append(pickle.load(fr))
    except EOFError:
        pass

info_df = pd.DataFrame(info, columns=colnames)
info_df
# info_df.to_csv('info.csv', index = False)

In [ ]:
# call exp pickle file
colnames = ['name', 'title', 'company', 'date_range', 'location', 'description']

import pickle
import pandas as pd
exp = []
with open('exp.pkl', 'rb') as fr:
    try:
        while True:
            exp.append(pickle.load(fr))
    except EOFError:
        pass

exp_df = pd.DataFrame(exp, columns=colnames)
exp_df